# 3-Dimensional Convolutional Neural Network (3D-CNN)

> To write this section, I have referred to the following resources: 
> - Niyas S. et al., *Medical image segmentation with 3D convolutional neural networks: A survey*.
> ---

While traditional CNNs are highly effective for processing 2D images, there are numerous applications where data inherently exists in three dimensions. Examples include medical imaging, video analysis, and 3D object recognition. To address these applications, **3D Convolutional Neural Networks** (3D-CNNs) have emerged as a powerful extension of the standard 2D-CNNs, capable of directly handling volumetric data and capturing spatial dependencies in three dimensions. 

At this point, you might be wondering, what is the difference between 2D and 3D CNNs? In the context of Convolutional Neural Networks (CNNs), images are typically represented as 3D tensors, where the dimensions correspond to height, width, and the number of color channels (e.g., RGB channels) while videos or volumetric data are represented as 4D tensors, where the additional dimension corresponds to time or depth. So, the primary difference between 2D and 3D CNNs lies in the convolution operation itself.

* The 2D convolution operation involves sliding a filter (kernel) over the height and width dimensions of the image. For example, the input 2D image can be represented as a 3D tensor with dimensions $(H, W, C)$ where $H$ is the height, $W$ is the width, and $C$ is the number of channels, the filter has dimensions $(f_H, f_W, C)$, where $f_H$ and $f_W$ are the height and width of the filter, respectively. At each position, the filter is multiplied element-wise with the input image patch, and the results are summed to produce a single output value. When the filter is convolved across the entire image, it produces a 2D feature map that captures spatial patterns in the image.

* In contrast, the 3D convolution operation extends this idea to three dimensions. Instead of sliding a 2D filter over the height and width dimensions of the image, a 3D filter is convolved across the height, width, and depth dimensions of the input volume. If the input volume has dimensions $(D, H, W, C)$ where $D$ is the depth, $H$ is the height, $W$ is the width, and $C$ is the number of channels, the 3D filter has dimensions $(f_D, f_H, f_W, C)$, where $f_D$, $f_H$, and $f_W$ are the depth, height, width of the filter, respectively, and the depth of the filter matches the depth of the input volume. At each position, the 3D filter is multiplied element-wise with the input volume patch, and the results are summed to produce a single output value. When the 3D filter is convolved across the entire volume, it produces a 3D feature map that captures spatial patterns in the volume.

However, I need to emphasize that the input data for 2D CNNs is not always 3D tensors. For example, in the case of grayscale images, the input data is represented as a 2D tensor with dimensions $(H, W)$ where $H$ is the height, $W$ is the width, and the number of channels is implicitly assumed to be 1. Similarly, in the case of videos, the input data is represented as a 4D tensor with dimensions $(T, H, W, C)$ where $T$ is the number of frames, $H$ is the height, $W$ is the width, and $C$ is the number of channels. But for grayscale videos, the number of channels is implicitly assumed to be 1 and it becomes a 3D tensor with dimensions $(T, H, W)$. 

The kernel always has the same number of channels as the input tensor. That why in all cases, we can say that the input of 2D CNNs is a 2D object and the input of 3D CNNs is a 3D object.

## Mathematical Formulation

### The Convolution Operation

From a mathematical perspective, the convolution operation is defined as the integral of the product of two functions after one is reversed and shifted. Formally, we write:

\begin{equation}
(f * g)(t) = \int_{-\infty}^{\infty} f(\tau)g(t - \tau) d\tau \tag{1.1}
\end{equation}

where $f$ and $g$ are the input function and kernel function, respectively, and $*$ denotes the convolution operation. 

In order to deal with discrete data, we can approximate the integral with a summation. Specifically, given two discrete functions $f, g: \mathbb{Z} \to \mathbb{R}$, the discrete convolution operation is defined as:

\begin{equation}
(f * g)(n) = \sum_{m \in \mathbb{Z}} f(m)g(n - m) \tag{1.2}
\end{equation} 

where $f$ and $g$ are the input function and kernel function, respectively and $n \in \mathbb{Z}$ is the index of the output.

Lastly, the convolution operation can be extended to higher dimensions. For example, we can write the convolution operation between two discrete and finite two-dimensional functions $f, g: \mathbb{Z}^2 \to \mathbb{R}$ as:

\begin{equation}
(f * g)(i, j) = \sum_{m \in \mathbb{Z}} \sum_{n \in \mathbb{Z}} f(m, n)g(i - m, j - n) \tag{1.3}
\end{equation}

where $(i, j) \in \mathbb{Z}^2$ is the index of the output.

Similarly, the convolution operation between two discrete and finite three-dimensional functions $f, g: \mathbb{Z}^3 \to \mathbb{R}$ can be written as:

\begin{equation}
(f * g)(i, j, k) = \sum_{m \in \mathbb{Z}} \sum_{n \in \mathbb{Z}} \sum_{p \in \mathbb{Z}} f(m, n, p)g(i - m, j - n, k - p) \tag{1.4}
\end{equation}

where $(i, j, k) \in \mathbb{Z}^3$ is the index of the output.

### The 3D Convolution Operation

In the context of 3D CNNs, the 3D convolution operation is defined as the element-wise multiplication of a 3D filter (kernel) with a 3D input volume, followed by the summation of the results. Formally, given a 3D input volume represented as a tensor $\mathbf{X} \in \mathbb{R}^{D \times H \times W \times C}$ and a 3D filter represented as a tensor $\mathbf{K} \in \mathbb{R}^{f_D \times f_H \times f_W \times C}$, after the convolution operation, we obtain a 3D feature map represented as a tensor $\mathbf{Y} \in \mathbb{R}^{D' \times H' \times W'}$ where $D'$, $H'$, and $W'$ are the depth, height, and width of the feature map, respectively. The 3D convolution operation can be mathematically expressed as:

\begin{equation}
\mathbf{Y}(i, j, k) = \sum_{m = 1}^{f_D} \sum_{n = 1}^{f_H} \sum_{p = 1}^{f_W} \sum_{q = 1}^{C} \mathbf{X}(i + m - 1, j + n - 1, k + p - 1, q) \cdot \mathbf{K}(m, n, p, q) \tag{1.5}
\end{equation}

where $(i, j, k) \in \mathbb{Z}^3$ is the index of the output feature map, and $(m, n, p) \in \mathbb{Z}^3$ is the index of the filter.

### The 3D Convolution Layer

In practice, a 3D convolution layer consists of multiple 3D filters, each of which is convolved with the input volume to produce a set of 3D feature maps. The output of the 3D convolution layer is a 4D tensor that contains the set of 3D feature maps. The number of filters in the 3D convolution layer is equal to the number of output channels. Assume that one 3D filter has dimensions $f_D \times f_H \times f_W \times C$ and the number of filters is $N$, then the output of the 3D convolution layer have shape $D' \times H' \times W' \times N$. Result of each convolution operation between the filter $f_D \times f_H \times f_W \times C$ and the input $D \times H \times W \times C$ is a 3D feature map $D' \times H' \times W'$. If we have $N$ filters, then the output of the 3D convolution layer is a 4D tensor $D' \times H' \times W' \times N$.

We define padding, stride, and dilation in the same way as in 2D CNNs.

1. **Stride**: Stride is the step size at which the filter is moved across the input volume. The stride size is usually set to control the spatial dimensions of the output feature map. If the stride size is $S$, then the spatial dimensions of the output volume can be calculated as:

\begin{equation}
D' = \left\lfloor \frac{D - f_D}{S} \right\rfloor + 1, \quad H' = \left\lfloor \frac{H - f_H}{S} \right\rfloor + 1, \quad W' = \left\lfloor \frac{W - f_W}{S} \right\rfloor + 1 \tag{1.6}
\end{equation}

where $S$ is the stride. In almost standard cases, the stride is set to $1$. Some architectures use a stride of $2$ to reduce the spatial dimensions of the output volume. They rarely use a stride of $3$ or more.

2. **Padding**: Padding is the process of adding zeros around the input volume to control the spatial dimensions of the output feature map. The padding size is usually set to ensure that the spatial dimensions of the input and output volumes are the same. If the padding size is $P$, then the spatial dimensions of the output volume can be calculated as:

\begin{equation}
D' = \left\lfloor \frac{D + 2P - f_D}{S} \right\rfloor + 1, \quad H' = \left\lfloor \frac{H + 2P - f_H}{S} \right\rfloor + 1, \quad W' = \left\lfloor \frac{W + 2P - f_W}{S} \right\rfloor + 1 \tag{1.6}
\end{equation}

where $S$ is the stride. In almost standard cases, the padding size is set to ensure that the spatial dimensions of the input and output volumes are the same. For example, if the padding size is $P = \frac{f_D - 1}{2}$ and the stride is $S = 1$, then the spatial dimensions of the input and output volumes are the same.

3. **Dilation**: Dilation is the process of adding zeros between the elements of the filter to control the receptive field of the filter. The dilation size is usually set to control the spatial dimensions of the output feature map. If the dilation size is $D$, then the spatial dimensions of the output volume can be calculated as:

\begin{equation}
D' = \left\lfloor \frac{D + 2P - f^{\text{eff}}_D}{S} \right\rfloor + 1, \quad H' = \left\lfloor \frac{H + 2P - f^{\text{eff}}_H}{S} \right\rfloor + 1, \quad W' = \left\lfloor \frac{W + 2P - f^{\text{eff}}_W}{S} \right\rfloor + 1 \tag{1.7}
\end{equation}

where $S$ is the stride and $f^{\text{eff}}_D = f_D + (f_D - 1) \times D + 1$ is the effective depth of the filter, similarly $f^{\text{eff}}_H$ and $f^{\text{eff}}_W$ are the effective height and width of the filter, respectively, they are calculated in the same way as $f^{\text{eff}}_D$. In almost standard cases, the dilation size is set to $1$. Some architectures use a dilation of $2$ or more to increase the receptive field of the filter.

Dilation increases the receptive field of a convolutional filter without increasing the number of parameters or the computational complexity. This means that the network can capture more context from the input image. A larger receptive field allows the model to integrate information from a broader area, which can be particularly useful for tasks requiring context from larger spatial regions.

### The 3D Pooling Operation

In the context of 3D CNNs, the 3D pooling operation is defined as the process of downsampling the input volume by selecting the maximum or average value within a local region. The 3D pooling operation is used to reduce the spatial dimensions of the input volume while retaining the most important features. The 3D pooling operation can be mathematically expressed as:

\begin{equation}
\mathbf{Y}(i, j, k) = \max_{m = 1}^{f_D} \max_{n = 1}^{f_H} \max_{p = 1}^{f_W} \mathbf{X}(S \cdot i + m - 1, S \cdot j + n - 1, S \cdot k + p - 1) \tag{1.8}
\end{equation}

where $(i, j, k) \in \mathbb{Z}^3$ is the index of the output feature map, and $S$ is the stride. The 3D pooling operation can also be performed using the average value instead of the maximum value. In this case, the 3D pooling operation can be mathematically expressed as:

\begin{equation}
\mathbf{Y}(i, j, k) = \frac{1}{f_D \times f_H \times f_W} \sum_{m = 1}^{f_D} \sum_{n = 1}^{f_H} \sum_{p = 1}^{f_W} \mathbf{X}(S \cdot i + m - 1, S \cdot j + n - 1, S \cdot k + p - 1) \tag{1.9}
\end{equation}

where $(i, j, k) \in \mathbb{Z}^3$ is the index of the output feature map, and $S$ is the stride.

### The 3D Batch Normalization Operation

In the context of 3D CNNs, the 3D batch normalization operation is defined as the process of normalizing the input volume by subtracting the mean and dividing by the standard deviation. The 3D batch normalization operation is used to stabilize the training process and improve the convergence of the network. Let's denote a 3D feature map $\mathbf{X}$ and $X$ has dimensions $N \times C \times D \times H \times W$, where $N$ is the batch size, $C$ is the number of channels, $D$ is the depth, $H$ is the height, and $W$ is the width. 

First, we calculate the mean and variance of the 3D feature map along the batch dimension:

\begin{align}
\mu_c &= \frac{1}{N \times D \times H \times W} \sum_{n = 1}^{N} \sum_{d = 1}^{D} \sum_{h = 1}^{H} \sum_{w = 1}^{W} \mathbf{X}(n, c, d, h, w) \tag{1.10} \\
\sigma_c^2 &= \frac{1}{N \times D \times H \times W} \sum_{n = 1}^{N} \sum_{d = 1}^{D} \sum_{h = 1}^{H} \sum_{w = 1}^{W} (\mathbf{X}(n, c, d, h, w) - \mu_c)^2 \tag{1.10}
\end{align}

where $\mu_c$ is the mean of the 3D feature map along the batch dimension, $\sigma_c^2$ is the variance of the 3D feature map along the batch dimension, and $c$ is the index of the channel. 

Next, we normalize the 3D feature map by subtracting the mean and dividing by the standard deviation:

\begin{equation}
\hat{\mathbf{X}}(n, c, d, h, w) = \frac{\mathbf{X}(n, c, d, h, w) - \mu_c}{\sqrt{\sigma_c^2 + \epsilon}} \tag{1.11}
\end{equation}

where $\hat{\mathbf{X}}$ is the normalized 3D feature map, $\epsilon$ is a small constant to prevent division by zero, and $n$ is the index of the batch. 

Finally, we scale and shift the normalized 3D feature map using learnable parameters $\gamma_c$ and $\beta_c$:

\begin{equation}
\mathbf{Y}(n, c, d, h, w) = \gamma_c \hat{\mathbf{X}}(n, c, d, h, w) + \beta_c \tag{1.12}
\end{equation}

where $\mathbf{Y}$ is the output of the 3D batch normalization operation, $\gamma_c$ is the scale parameter, and $\beta_c$ is the shift parameter. The scale and shift parameters are learned during training using backpropagation.


### The 3D Activation Function

In the context of 3D CNNs, the 3D activation function is defined as the process of introducing non-linearity into the network. The 3D activation function is used to introduce non-linearities into the network and allow the network to learn complex patterns in the data. The 3D activation function can be mathematically expressed as:

\begin{equation}
\mathbf{Y}(i, j, k, c) = \sigma(\mathbf{X}(i, j, k, c)) \tag{1.13}
\end{equation}

where $(i, j, k, c) \in \mathbb{Z}^4$ is the index of the output feature map, $\sigma$ is the activation function, and $\mathbf{X}(i, j, k, c)$ is the input volume. The activation function is usually applied element-wise to each element of the input volume. Common activation functions used in 3D CNNs include the Rectified Linear Unit (ReLU), the Sigmoid function, and the Hyperbolic Tangent function. The ReLU activation function is the most commonly used activation function in 3D CNNs due to its simplicity and effectiveness. The ReLU activation function can be mathematically expressed as:

\begin{equation}
\text{ReLU}(x) = \max(0, x) \tag{1.14}
\end{equation}

where $x \in \mathbb{R}$. The ReLU activation function sets all negative values to zero. This allows the network to learn complex patterns in the data and improve the performance of the network. The ReLU activation function is usually applied after the convolution operation and before the pooling operation. 

### The 3D Fully Connected Layer

Similar to 2D CNNs, 3D CNNs can also have fully connected layers at the end of the network. The fully connected layer is used to combine the features learned by the convolutional layers and make a final prediction. Before going into the fully connected layer, the 3D feature maps are flattened into a 1D vector. The fully connected layer consists of a set of neurons that are connected to all the neurons in the previous layer. The fully connected layer can be mathematically expressed as:

\begin{equation}
\mathbf{Y} = \sigma(\mathbf{W} \cdot \mathbf{X} + \mathbf{b}) \tag{1.15}
\end{equation}

where $\mathbf{Y}$ is the output of the fully connected layer, $\sigma$ is the activation function, $\mathbf{W}$ is the weight matrix, $\mathbf{X}$ is the input vector, and $\mathbf{b}$ is the bias vector. The weight matrix $\mathbf{W}$ is learned during training using backpropagation. The fully connected layer is usually followed by a softmax activation function to convert the output into a probability distribution over the classes in classification tasks.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# Implement a 3D CNN using PyTorch
class CNN3D(nn.Module):
	def __init__(self, in_channels, num_classes):
		super(CNN3D, self).__init__()
		self.conv1 = nn.Conv3d(in_channels, 8, kernel_size=(3, 3, 3), padding=(1, 1, 1))
		self.conv2 = nn.Conv3d(8, 16, kernel_size=(3, 3, 3), padding=(1, 1, 1))
		self.conv3 = nn.Conv3d(16, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1))
		self.conv4 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
		self.conv5 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
		self.fc1 = nn.Linear(128, 1024)
		self.fc2 = nn.Linear(1024, 512)
		self.fc3 = nn.Linear(512, num_classes)
		self.pool = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
		self.dropout = nn.Dropout(p=0.5)
		self.relu = nn.ReLU()

	# Assuming input shape is (N, C, 32, 32, 32)
	def forward(self, x):
		x = self.relu(self.conv1(x)) # (N, 8, 32, 32, 32) due to padding
		x = self.pool(x)             # (N, 8, 16, 16, 16) due to pooling
		x = self.relu(self.conv2(x)) # (N, 16, 16, 16, 16)
		x = self.pool(x)             # (N, 16, 8, 8, 8)
		x = self.relu(self.conv3(x)) # (N, 32, 8, 8, 8)
		x = self.pool(x)             # (N, 32, 4, 4, 4)
		x = self.relu(self.conv4(x)) # (N, 64, 4, 4, 4)
		x = self.pool(x)             # (N, 64, 2, 2, 2)
		x = self.relu(self.conv5(x)) # (N, 128, 2, 2, 2)
		x = self.pool(x)             # (N, 128, 1, 1, 1)
		x = x.view(-1, 128)          # => need to flatten to (N, 128)
		x = self.relu(self.fc1(x))
		x = self.dropout(x)
		x = self.relu(self.fc2(x))
		x = self.dropout(x)
		x = self.fc3(x)
		return x

In [3]:
model = CNN3D(in_channels=1, num_classes=10)
x = torch.randn((1, 1, 32, 32, 32)) # (batch, channel, depth, height, width)

model(x).shape

torch.Size([1, 10])

In [4]:
'''
Above is simple 3D CNN model with 5 convolutional layers and 3 fully connected layers.
We can define more complex model with more layers and different hyperparameters.
	- In some state-of-the-art models, they use BatchNorm and LeakyReLU instead of Dropout and ReLU.
	- Also, they add some skip connections to improve the performance and ignore the degradation problem.
'''

# Implement a 3D CNN using PyTorch
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.downsample = None

        # This is used to adjust the dimensions of the residual block
        if in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(out_channels),
            )        

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out